In [ ]:
import pandas as pd
import os
import sys
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.stats.diagnostic import acorr_ljungbox
import warnings 
warnings.filterwarnings("ignore")
f=input()
fe=os.path.join(sys.path[0],f)
df=pd.read_csv(fe)
df['Date']=pd.to_datetime(df['Date'])
df.set_index(df['Date'],inplace=True)
series=df['Close_diff'].dropna()
size=int(len(series)*0.8)
train=series[:size]
test=series[size:]
print("Training data size:",train.shape[0])
print("Testing data size:",test.shape[0])
print()
best_p=0
aic_values={}
best_aic=np.inf
best_model=None
best_type=None
best_order=None
for p in range(1,6):
    model=ARIMA(train,order=(p,0,0))
    result=model.fit()
    aic=result.aic
    aic_values[f"AR({p})"]=aic
    print(f"AR({p}) AIC: {aic}")
    
    if(aic<best_aic):
        best_aic=aic
        best_model=result
        best_type="AR"
        best_order=p
    
for q in range(1,6):
    model=ARIMA(train,order=(0,0,q))
    res=model.fit()
    aic=res.aic
    aic_values[f"MA({q})"]=aic
    print(f"MA({q}) AIC: {aic}")
    if aic<best_aic:
        best_aic=aic
        best_model=res
        best_type="MA"
        best_order=q
print(f"Best Model: {best_type}({best_order})")
print(best_model.summary())
print()
residual=best_model.resid
ljung=acorr_ljungbox(residual,return_df=True,lags=[1])
print(f"Ljung-Box test Results:")
print(ljung)